In [1]:
# Dependencies and Setup
%matplotlib inline
from config import gkey
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gmaps
import json
import scipy.stats as st
import requests
from scipy.stats import linregress
from sklearn import linear_model

# Configure gmaps
gmaps.configure(api_key=gkey)

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')

# Files to Load
potholes = pd.read_csv("Resources/311_Service_Requests_-_Pot_Holes_Reported_-_Historical-2016.csv")
crime = pd.read_csv("Resources/Crimes_-_2001_to_present(2016).csv")
parks = pd.read_csv("Resources/CPD_Parks.csv")
socio = pd.read_csv("Resources/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv")

In [2]:
#MK dfs

#pothole df
cleanpotholes=potholes.dropna(subset=['NUMBER OF POTHOLES FILLED ON BLOCK', 'LATITUDE', 'LONGITUDE'])
potlocations = cleanpotholes[["LATITUDE", "LONGITUDE"]]
potholenumber = cleanpotholes["NUMBER OF POTHOLES FILLED ON BLOCK"].astype(float)

#parks df
parksdf = pd.DataFrame(parks)
cleanparks = parks[['PARK', 'LOCATION', 'ZIP']]
cleanparks["Lat"] = ""
cleanparks["Long"] = ""
cleanparks.head()

,PARK,LOCATION,ZIP,Lat,Long
0,MCGUANE (JOHN),2901 S POPLAR AVE,60608.0,,
1,ARMOUR (PHILIP) SQUARE,3309 S SHIELDS AVE,60616.0,,
2,FULLER (MELVILLE),331 W 45TH ST,60609.0,,
3,CORNELL (PAUL) SQUARE,1809 W 50TH ST,60609.0,,
4,RUSSELL (MARTIN) SQUARE,3045 E 83RD ST,60617.0,,


In [3]:
params = {"key": gkey}

# Loop through the df and run a lat/long search for each park
for index, row in cleanparks.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    address = row['LOCATION']
    zipcode = row['ZIP']

    # update address key value
    params['address'] = f"{address} Chicago {zipcode}"

    # make request
    parkscoordinates = requests.get(base_url, params=params)
    
    # convert to json
    parkscoordinates = parkscoordinates.json()

    try:
        cleanparks.loc[index, "Lat"] = parkscoordinates["results"][0]["geometry"]["location"]["lat"]
        cleanparks.loc[index, "Long"] = parkscoordinates["results"][0]["geometry"]["location"]["lng"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
# Visualize to confirm lat lng appear
cleanparks.head()

Missing field/result... skipping.


,PARK,LOCATION,ZIP,Lat,Long
0,MCGUANE (JOHN),2901 S POPLAR AVE,60608.0,41.8407,-87.6476
1,ARMOUR (PHILIP) SQUARE,3309 S SHIELDS AVE,60616.0,41.8336,-87.634
2,FULLER (MELVILLE),331 W 45TH ST,60609.0,41.8123,-87.6346
3,CORNELL (PAUL) SQUARE,1809 W 50TH ST,60609.0,41.8025,-87.6701
4,RUSSELL (MARTIN) SQUARE,3045 E 83RD ST,60617.0,41.7439,-87.55


In [4]:
#MK define parks locations

cleanparks["Lat"] = pd.to_numeric(cleanparks["Lat"], errors='coerce')
cleanparks["Long"] = pd.to_numeric(cleanparks["Long"], errors='coerce')
cleanparks2 = cleanparks.dropna(subset=['Lat', 'Long'])
parkslocations = cleanparks2[["Lat", "Long"]]

In [8]:
#MK heatmap
fig2 = gmaps.figure()

# Create heat layer
heat_layer2 = gmaps.heatmap_layer(potlocations, weights=potholenumber, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=.0015)
#create symbol layer
symbol_layer = gmaps.symbol_layer(parkslocations,  
                                   fill_color=None, fill_opacity=1.0, stroke_color=None, 
                                   stroke_opacity=1.0, scale=3, info_box_content=None, 
                                   display_info_box=None)

# Add layer
fig2.add_layer(heat_layer2)
fig2.add_layer(symbol_layer)

# Display figure
fig2

Figure(layout=FigureLayout(height='420px'))